In [102]:
import pandas as pd
df = pd.read_csv("Desafio.csv")

num = display(df)

,id_venda,id_loja,data_venda,pais,id_produto,quantidade
0,271783,12,2022-08-04,Alemanha\r,231,10
1,271784,20,2024-04-10,Japão\r,127,10
2,271785,17,2022-10-03,Brasil\r,380,4
3,271786,6,2022-01-14,Singapura\r,931,9
4,271787,16,2023-10-25,Indonésia\r,381,9
...,...,...,...,...,...,...
271776,543559,15,2023-06-24,Canadá\r,504,5
271777,543560,9,2022-06-25,Singapura\r,736,5
271778,543561,17,2022-02-09,Brasil\r,606,2
271779,543562,4,2022-05-05,Japão\r,92,8


In [103]:
import ephem
import pycountry
from datetime import datetime
from geopy.geocoders import Nominatim

def estacao_do_ano(pais_nome, data):
    # Usar geopy para obter as coordenadas do país
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(pais_nome)
    if not location:
        raise ValueError("País não encontrado")

    # Determinar hemisfério
    latitude = location.latitude
    hemisferio = 'N' if latitude >= 0 else 'S'

    # Calcular a estação do ano
    mes = data.month
    if (mes >= 3 and mes < 6):
        estacao = "Outono" if hemisferio == 'S' else "Primavera"
    elif (mes >= 6 and mes < 9):
        estacao = "Inverno" if hemisferio == 'S' else "Verão"
    elif (mes >= 9 and mes < 12):
        estacao = "Primavera" if hemisferio == 'S' else "Outono"
    else:
        estacao = "Verão" if hemisferio == 'S' else "Inverno"

    return estacao

# Exemplo de uso
pais_nome = df["pais"][i][:-1]
data = datetime(2024, 7 , 16)
estacao = estacao_do_ano(pais_nome, data)
print(f"A estação do ano em {pais_nome} em {data.strftime('%d/%m/%Y')} é: {estacao}")


A estação do ano em Quênia em 16/07/2024 é: Verão


In [104]:
def datetransform(data):

    ano, mes, dia = data.split("-")
    
    if mes[0] == "0":
        mes = mes[1]
    
    if dia[0] == "0":
        dia = dia[1]


    newdate = [ano, mes, dia]
    
    return newdate


print(datetransform(df["data_venda"][2]))

['2022', '10', '3']


In [105]:
dado = {
    "id_item" : [],
    "primavera" : [],
    "verao" : [],
    "outono" : [],
    "inverno" : []
}

for i in range(0, len(df)):
    dado["id_item"].append(df["id_produto"][i])

    pais_nome = df["pais"][i][:-1]

    ano, mes, dia = datetransform(df["data_venda"][i])

    ano = int(ano)
    mes = int(mes)
    dia = int(dia)
    
    data = datetime(ano, mes, dia)
    estacao = estacao_do_ano(pais_nome, data)

            
    if estacao == "Primavera":
        dado["primavera"].append(df["quantidade"][i])
        dado["verao"].append(0)
        dado["outono"].append(0)
        dado["inverno"].append(0)


    if estacao == "Verão":
        dado["primavera"].append(0)
        dado["verao"].append(df["quantidade"][i])
        dado["outono"].append(0)
        dado["inverno"].append(0)


    if estacao == "Outono":
        dado["primavera"].append(0)
        dado["verao"].append(0)
        dado["outono"].append(df["quantidade"][i])
        dado["inverno"].append(0)


    if estacao == "Inverno":
        dado["primavera"].append(0)
        dado["verao"].append(0)
        dado["outono"].append(0)
        dado["inverno"].append(df["quantidade"][i])



dataframe = pd.DataFrame(dado)

display(dataframe)

GeocoderInsufficientPrivileges: Non-successful status code 403

In [13]:
print(len(df['pais']))

271781
